In [1]:
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline

In [3]:
import numpy as np 
import pandas as pd 
# Read the CSV file, providing the correct file path
file_path = "training.1600000.processed.noemoticon.csv"
df = pd.read_csv(file_path, encoding="ISO-8859-15", names=['Target', 'ID', 'Date', 'Flag', 'User', 'Text'])

# Display the first few rows of the dataframe
df.head()

,Target,ID,Date,Flag,User,Text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [4]:
print("Target values:\n"+str(df['Target'].unique()))
print("\nTarget counts:\n"+str(df['Target'].value_counts()))
print("\nCount of null values per column:\n"+str(df.isnull().sum()))

Target values:
[0 4]

Target counts:
Target
0    800000
4    800000
Name: count, dtype: int64

Count of null values per column:
Target    0
ID        0
Date      0
Flag      0
User      0
Text      0
dtype: int64


In [5]:
X = df['Text'].values
Y = df['Target'].values

In [6]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.20, random_state=32)

In [7]:
vect = CountVectorizer()
vect.fit(x_train)

X_train = vect.transform(x_train)
X_test = vect.transform(x_test)
#print(x_train[4])
#print(X_train[4])

In [8]:
print(X_train[0])

  (0, 56129)	1
  (0, 57447)	1
  (0, 126962)	1
  (0, 129411)	1
  (0, 132556)	1
  (0, 142883)	1
  (0, 226884)	1
  (0, 252021)	1
  (0, 256925)	1
  (0, 257741)	1
  (0, 257883)	1
  (0, 433510)	1
  (0, 455368)	1
  (0, 467486)	1
  (0, 486108)	1
  (0, 501574)	1
  (0, 517177)	1
  (0, 520059)	1
  (0, 528806)	1
  (0, 558368)	1
  (0, 562011)	1
  (0, 562867)	1
  (0, 566030)	1
  (0, 566811)	1


In [9]:
symbols = ",.!?/&-:;@'..."
"["+"\\".join(symbols)+"]"

s = x_train[0]
s = ' '.join(x for x in re.split("["+"\\".join(symbols)+"]", s) if x)

for x in s.split():
    if len(x) > 1:
        print(f"{x : <10}{vect.vocabulary_[x.lower()] : ^10}")

It          257883  
was         558368  
rainy       433510  
and         57447   
cloudy      129411  
in          252021  
the         517177  
Windy       566030  
City        126962  
today       528806  
amp         56129   
WF          562011  
customers   142883  
had         226884  
some        486108  
serious     467486  
SAD         455368  
issues      257741  
with        566811  
them        520059  
when        562867  
is          256925  
summer      501574  
coming      132556  


In [10]:
classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train, y_train)

C:\Users\chaud\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000)

In [11]:
unique = df['Target'].unique()
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred, labels = unique)

df_cm = pd.DataFrame(cm, index=unique, columns=unique)
df_cm

,0,4
0,126497,33824
4,30083,129596


In [12]:
NB = make_pipeline(CountVectorizer(), LogisticRegression())
NB.fit(x_train, y_train)
score = NB.score(x_test, y_test)
print(score)

C:\Users\chaud\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.79830625


In [13]:
unique = df['Target'].unique()
y_pred = NB.predict(x_test)
cm = confusion_matrix(y_test, y_pred, labels = unique)

df_cm = pd.DataFrame(cm, index=unique, columns=unique)
df_cm

,0,4
0,125633,34688
4,29854,129825


In [14]:
NB = make_pipeline(TfidfVectorizer(), LogisticRegression())
NB.fit(x_train, y_train)
score = NB.score(x_test, y_test)
print(score)

unique = df['Target'].unique()
y_pred = NB.predict(x_test)
cm = confusion_matrix(y_test, y_pred, labels = unique)

df_cm = pd.DataFrame(cm, index=unique, columns=unique)
df_cm

C:\Users\chaud\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8025125


,0,4
0,127017,33304
4,29892,129787
